# Reprojecting to EPSG:4326

The same place on earth can be described in different ways depending on your coordiante reference system and map projection.  Sometimes you have a location description in one system but you would like to convert it to another.  This process is called reprojecting and it is the subject of this noteook.

## Vectors and Rasters

The two main sections of this notebook are:
1. Reprojecting Vector Objects
2. Reprojecting Raster Objects

To recap the difference, vector objects are a single point or a series of points that make a shape, whereas raster data is the pixel structure discussed in other notebooks.

In [1]:
from IPython.display import Image

In [2]:
Image(url= "data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUTEhMWFRUXFxsbGBgYFxgaHRcdGBoaGBoYHhoYHSggHR0lIB0YITEhJSktLi4uHR8zODMtNygtLisBCgoKDg0OGxAQGzcmHyY2LysrLy0rLS81LS0tLjUtLi0tLTI3Ly0vLTctLS4tLS0tLS0vLS8tLS0tLS0tLS0tLf/AABEIAJ8BPQMBEQACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAABQQGAgMHAQj/xABQEAABAgMEBAcKCgkCBgMAAAABAgMABBEFEiExBhMiQRQjMjNRU2EHJEJDUmJxc4G0FTRUY5GhorHR0kRydJOUsrPB0xbhZIKSwvDxF6Pi/8QAGgEBAAIDAQAAAAAAAAAAAAAAAAEFAgQGA//EADsRAAIBAgQEAwUGBQMFAAAAAAABAgMRBBIhMQVBUWETInGBkaHB0RQyUmKx4RUjMzRCJHKiBlOSwvD/2gAMAwEAAhEDEQA/APNONK59qefbamXEoS5spF2gFBlUQAgXpzaIV8ccpTLZz/6YA2Maa2kRUzbv2fwgDejS+0jXvp37P4QAumdO7TB+OOj/AKfywBp/1/afyx37H5YADp9afyx37H5YA9Vpzag/TXfsflgDD/Xtp/LXfsflgDJGnlpk/HXfsflgCS/p5aIGE479n8sAeymnFpEYzjv2fywB9BaOPKXKS61kqUphtSicyShJJ9pgDXpM0lTISpIUkvy4IIBB74awIMQzKO5s+AZX5Mx+6R+ETYjMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMw+AZX5Mx+6R+ELDMyBbVjy6EIUhhpKg/L0KW0AjvhrIgRBKbLBEmIo0l1VxrX3NXrk3tZduZKpW9hnTPfSAEs6bL1jFOA0vqvU1GWrXSvtpAGDxsvhTVOBXdS9XmKVvy9K7q0vU9sAZNmy+EufEbuqbpzFK33a+3L6oA9ljZeuerwGlEU5imRrSAPLN+C9ZM14FTXC7XUUpqGcuytfbWAPLDFmatV7gVddMZ6nLXuXc91KU7KQFj2yxZnB0XuBXru/UV3wFiHNizfg1dOB6zgh6m9e1X01rAWJ1siy9Xs8CrrGstRlrUV9lKwFjy0zZdG7vAedRWmoyrj7IAwtI2XrJa7wKmuN6moy1D2fZW77aQBVO6eZDvfU8F8Zeuar5ulbvtgCr90OVdcn5ooTUJdoThhsjpgCFYlhJcSsuIOygkEKwJBIIz9EALHU3VFIyEAS5Fu9hXChNOkjdAGpvR9+YJLaaJBpUkZjdie2AN6dBpjeQPo/NAHv8AoiY6R9n80ASEaGPXRtJPbh+MARntCHwCq8MPR+MAVwmlUncYAyDlRSntgCcwigoDWAPpnRX4lK/s7X9NMAZaQc0n18v7w1AmIygQEAEAEAc9ne6E4hxxCWEEJWpIJUqpCVEA5dkVFXimSbjl27lFW4z4dRwybO25qHdId+To/wCtX5Yw/i/5fiYfxz8nxPf/AJIc+To/eH8sP4v+X4j+Ofk+J6nukOb5dPscP5Yn+L/l+I/jn5PiSJXuiFS0IMuBeWlNb+V4gV5PbGdPiinJRy79z1pcZVSajk37l9i2LsIAW6Qc0n18v7w1AmIygQLLbeCNSpRoA8muBPgrGQgCJP2uzrZfaODivAX1TnmwBg/azXC2TeNAw/4C965bsgDJq12eFOG8aalrwF+W92QB7K2uzr3jeOTfgL6FebAHLtL51BtCaIJoVt02VfJ2R0Rz/EoN1tOiOX4vTlLEXXRCKSmkXTj4bm49YrsjTqU5X9i/RFfWpSzbcl0/CglZpGrGO7oP4QnTlnYqUpeI9DS7NJ4MRXxJ3HyPRGcacvGXr8z0jSl9oTt/l8xrZ84jXsGuUwyclbnkHojLBwkq8TPAU5LEQbXM7BatrskN0UeeR4C+n9WOnOxNdqWs0XZXaOD6q7C/k746IAqHdYtFtXBrqiaa3wVDq+kQBWdKJlTc/PLEvr0hYrU0CKAGsAUSbnSpwqCboJrdGQqawBsTM3yfQIAktrISSOio7DAFw0GWVS5Ksy4r7kwBZAIA2sLukK6IA0Ts0hG0shIJ+vOAMXTskjHCAOPONAqJ6VHD2wBrBobsASmTTGAPprRQ1kZU/wDDtf00wBnpBzSfXy/vDUCYjKBAQAQAQBwWaVVxZ6VqP0qJjka/9SXqzhcS71perNceR4hABAEuxvjDHrm/50xsYX+tH1NrBf3EPVHc46s7YIAW6Qc0n18v7w1AmIygQLLcdCNSohRAeTglJUeSsYJSCTAEOftZGtl9h7BxXiHuqc8yAMH7VRwtk3HuYf8AEPVxXLbrnZAGTVrI4U4bj3MteIe8t7dcgD2VtZGveNx7JvxD3QrzIA5dpfOpM/NEJcxW3m24D8XZGIKaiOf4lBut7FzRy/F4N4i+my5oRyUyLpwXy3PAX1iuyNOpTd+Wy5roivrUnm3W0ea/CglZkasYLy8hf4QnTed7e9E1KT8R6r3o0uzI4MRRfMnwF+R00jONN+Mtt+q6mcab+0J3X3uq6jSz5sa9g3V4TDJ5te51HZGWDg1Xi9PejPAU2sRB6b9UdgtW1kEN7D3PI8Q90/qR052JrtS1kF2V2HsH1eIe+TvjDYxgCod1i0Eq4NRLopreU06nq/KSKwBRu6JPuNz82lCiErc2gKY7IgCmh3sxgCTJoOZ3wBOZRe2emALxoe3Rig8tX9oAe0NDTOAIdjKdCCH1XlBahWgFRhTAQApteRU4p4PmqEtlbe66akbs8IAeNo4sfqj7oA4+64Nob7x++AGdhySXHdvABBND4Wyd/wBcARVClYA+l9EviMp+zs/00wBs0g5pPr5f3hqBMRlAgIAIAxdVRJPQDBkPY4Is1JPSY4+o7zb7nCVXepJ92eRgeYQAQBOsD41L+vb/AJxGzg/68fU28B/cw9TuEdUdqEALdIOaT6+X94agTEZQIFduuFIZUEKWQ8nZTdBOyrK+pKe3EiAIM/aTmtl+83xRavClseKX8/AGD9oucLZPBH+YfwvS1TVctjz1MP7iAM2rSc4S4eBv8y1helq8t7Hn6f8AqAPZW0nNe8eBv5N4XpbDA/PwBy/S6aWZ+aJYcSStGBLVRxDIxo4R24HfHP8AEop1tWlouv0OX4vBPEayS0XX5JiOSfVdPFL5bm9vrFefGnUgr/eWy69F2NCtTjm+8tl+L8K7BKvq1Y4peXS3+eJnBZ35l8foKlOPiPzL/l9DS6+rgx4pfNHGrfkZ8usZRgvGXmW/fr6HpGEftC8y+936+g0s+YVr2OJcPHs4Vax41GHOZmM8HBKvHzL4/QzwEIrEQ8y37/Q69atpOEN95vjjUeFLdOWD8dMdga7UtJzWSvej4o+rwpbHvd8UFH/bj0QBUe6tOrVwass6imt5RYx5vK66frgChd0mZpaE0mleM/7RAFTDorWkATJVysAO7NTXeMEKPoxEAWzRAcR/zq/tAD1CqGsAaHHFXwAnClSa7/RAGNqKq05hU3DAGxs7A/VH3QBx4oSSsk41OHtgCx6OpJlJl0jIXE/Qn80AJ5aldqAPpPRX4lK/s7X9NMAZaQc0n18v7w1AmIygQEAEAaJ5VG1noQr7jES2ZjLZnBkZD0Rxz3OCe5lEEBABADHRwVm5f1yPqNY2sF/XibnD/wC5h6nbY6k7QIAW6Qc0n18v7w1AmIygQLLcUoakoSFK1yaAm6Dsr30NPogCHPvzGtl+JRziqcceqc+bgDB9+Y4WzxKK6h/DWny5auOr9EAZtPzHCnOJRXUteOPlvfNwAmtPS0ysy4lyXqpSW1bLgIA2hmUjojVxGMp0GlK+vQ0sVj6WGkozvrroc9tq1FTE1MPJboFrTgV4i6y0ncOyKXG1Kdaamnuund9znuIVqdeoqibSa6d33Fsktd07A5bnh/OK82NeooX35Ll2Rq1lDNvyjy/Ku4Sq16sbAy8v/wDMJqGd6/AmooeI9fh+5pdWvgx2BTUnwvM/VjOKj4y159O5nFQ+0LX/AC6d/Ua2e45r2NhPxhnw/nUebGWDUfHjZ/AzwCh9ohZ8+n7nX7VfmKN1ZRzyPHHp9XHTnYmFqPzGtlaso59VOOOPe7/zeG+AKf3WHXjwa80lPO0o5Xq/MEAc87ozd605v1n/AGpgCtmVwr2wASywFJryQakdI6IAtE9bzAaBbb4wpu5nAe0Y5QBYtB1kywUd6lH66QA/XWhCRVVMBlAESynXSlQfTcWlRBFQcMOiAIk06/eeSsAN6slBqOmnpgBm2dgfqj7oA44EAuKB6T98ATOGq1WqB2L1fu/AQBGbdrWAPp3RP4jKfs7P9NMAZ6Qc0n18v7w1AmIygQEAEAQLfeCJZ9RNKNL/AJTHnVllg32PKtLLTk+zOHiOQOEPYAIAIAa6KJrOy/rB9QJjbwH9eJvcNX+pidpjqDsggBbpBzSfXy/vDUCYjKBAsty9xNwAq1yaXiQOSvMgEwBEn1TOtl6pZ5xVNtfVOeZAGt9UxwtnZZrqH6ba6cuWr4HogDNpUzwpzZZrqWvDX5b3mQBzzuhF7hpvBuuqbyUojNfSmKPiuXxI36HOcay+LG/T5lVly5ecwRyxvPkI7Ir55LR32+bKqp4eWO+3bqwki5dOCOW5vPWK7IVMl+ey/RCt4ebnsun4UEqXNWMEZdJ/CE8md7k1PD8R7/A0ulzgxwRTUnefI9EZxyeMt9/mZxyfaFv975jWzy5r2MEfGGaYqz1yKboyweTx42uZ4DJ9oha+51+1VTNG6pZ55Hhr6f1I6c7EwtRUzrZWqWefVTbX8nfz2PTAFP7rCn+9r6WxztKKUer6UiAKZpw8E2jNYDnP+0QAmt+cQEpabSBspUpQOZINR9YgCvqVAE6wZAzDyWxlgVfqgisAdZlGEoSEJFABQCANba1odWpR2bguimWPTvgBA7pLM1Peyse0Y9uUAAnZt7ZU1cChQkkYCALMUkJu9lIA466zVxSR0n74A8dG6kASGGrqcRiYA+mdFPiUr+ztf00wBnpBzSfXy/vDUCYjKBAQAQBQO6ZbIomWQoHG85Q5U5KT7amnYIqeKV7R8Nc9yj4zicsFSi99ygRRHNhABABADjQ0Vnpf9c/UhRjd4f8A3ESw4X/dR9v6HZo6Y7AIAW6Qc0n18v7w1AmIygQK7dCyGQ2pKVa5NCpJUBsqzSFJJw7RAEGfZm9bL1fYrfVTvdzDil/8RjAGD7M3wtnjmK6h+h4O5Sl+Wrhwj0b+mAEOlNuTknMi6uXWVtJrVhwABK10w1/afqjSxuLeHy2V73K7iGOeFy2je9/hb6lGte1ZiafU44poKCUJ2WlgUF4jAunHHpinxOJVe0pR9z/YocZi44jLOce2j/Zi6XQ7ec20cseLV5CPnI8JuFo6Pbr3fY16jp5Y6Pbqur7BJIdunbRy3PFq6xXzkKjp32ey59l2FZ082z2XNfhXYJVDurG2jLq1f5Imbhnej9/7Co6fiPR+9fQ0upc4MdtFNUfAV5HTfjKLh4y0e/Xv6HpF0/tC0f3uq6+g0s9DuvYotuuvZpxas9ainjMR2Rng3Dx42T9/7GeAdP7RCye/VfQ69arM3Rur7HOo/R3M6/tEdMdga7UZm9ZK1eY59VO93MDwd/PvjHCvRAFR7qzUwODX3Wlc7S6ytNObrWrxrAFM7olnqXNzbtRdS/d9txJgCrIlajPLMdkAS7FsBcys3dlsHFR9lQMYA6BYlhNSyaNjaOascfpMAMVqABJNKDPo7YAUDShkOBs4jK9QgenL0wBYkLoIA8WqsAR3E1BHZAHG01vqPafvgD1kFSgc4AlpXhAH0ror8Slf2dr+mmAMtIOaT6+X94agTEZQIMHnUpSVKISkCpJNAB6Yhu2rIbSV2c70n07K6tyhKU5F3In9Qbh5x9nTFRi+JW8tL3lFjeLJeSj7/oUj7zmd5PSTvMUrk5O7OelJyd2EQQEAEAEAOtCR3/L/AKyv6a43uHf3C9v6Fjwr+6j7f0OyR0p14QAt0g5pPr5f3hqBMRlAgWW42VakBRQS8naTdJGyrK8COzEQBCn5BzWy/fT3OK8FjDil/NQBXtOJyYlHpdbcwtSlNvjbQyaC8wcLrY7PojTxuJlQgpRXO2pX8Rxc8NTUoJO7tr6FFti1H5p6rrpqlsAUS2M1K82KbEYuVaKc4rS/Xt3KDFY6WIhFzitG+vbuLm2VX18YrwdyOg+bHg5xyryrn1+prSnHJHyrn1+p5LsqvOcYrljcjyEebCco2j5Vt36vuTUnHLHyrbv1fcJJlV08YrlubkdYrzYVJRv91bLr0XcVpxzfdWy6/hXcJVlWrHGKy6EflhOUc78q+P1FScfEflXx+pqdZVwY8YrmjhRHkZcmM4yXjLy8+/X1M4zj9oXlX3u/X1GlnsK17HGr+MM40bw41GPIjLByTrx8v6/UzwE4vEQ8q37/AFOv2rIOUb76ePHI8Fjp9VHTnYmFqSDmtle+njV9XgsYd7v4817PbAFP7q8otPBqzDi+d5SWcOb8lsQBW+6i0rWPEGideajpNwQAm0N0fS8Nc6bwBoEdooakg9sAX1psJASkUA3QBCtK2GmClKySTTAA7zSuAgCYtAWnEXkrGWWcAR5+wWnUFCkAYYKFcBTAZ+mAIejE6o35Z3nGiQD5SBgk4QA9UmANLowNeiAOQpbwPSSYAdWNZRLaHEitFlKx/wCekQBFm7NKRtml7LCvbugD6I0S+Iyn7Oz/AE0wBs0g5pPr5f3hqBMRg4ogEgVIGXT2QIZxvSLSJ6aUQ5sIBwaGQIw2vKUPqjm8ZjKlSTjsuhyOOx9WrJweiXITxoFaEAEAEAEAEAPdBU1n2ey+fsKH943+Gq9dFnwlXxKfqdhjpDrQgBbpBzSfXy/vDUCYjKBAsttu9qU1KavJxSaEbK8jAFY7oJcl22HGn3QrXEVKgcC05XAjsjUx1aVKlmhuaPEa86NDPDe6Od2taD8w42HnlruocpW6KVLVcgOyKSpi6lWn59bNcuzOdq42rWpPxLOzXLsyEmX4xW0vkJ39q48HPyLRbv5Gu6n8taLd8uyBuX217a/B39hg5+VaLmJVPJHRc+R5Ly+05tL5Y3+YiJnPSOi2+bFSp5Y6Lbp3YSTGydpfLc3/ADioVJ67LZfohWqebZbR5flQSrHFjaXl0wnPzvRCpU/mPRe40usd7E3lcyd/mRnGf85aLf5npGf+oWi+98xrZ8tx7G2v4wz4XzyOyMsHO9eOiM+HzviIaLc7BatnUDfHPc8jw+30R052JrtSzqOyvGvYvq8PLvd/LCAKh3WJK7wbjHDzuaq9X2QAk03mZdcy828VbL5VQA47KRugBBY0+0zNhLKiWnABiCKKJpv9EAX1KYAxmGmwCtScaYmAI63FKBUyQaAhI7YAUImXWHCt5zWFaBRCRkqpNMBlSmMATrOlFqd4Q6AlVy6ADkK3t0AMpqaQ2Ly1XR7f7QBULa0mWsXWhdR5RpU57iPRAFZoAcYAvOhCay6ifLNPRRMARrYlXFNqDQBVhSpA3g74A7Ho8kiVlwcwy3X03BWAMdIOaT6+X94agTEZQIOZd0aw9U7wlA2HDRfmr6f+b7x2xScTw1n4sfac7xjCWfjR9pTopyhCACACACAHNgaMvzRBQLjW9xQw/wCUZqP1dsb2GwFSrq9EWWE4ZVr6vSJ02wNG2JUcWmqyMXFYqPt3DsEX1HD06KtFHS4fCU6CtBe0cR7myEALdIOaT6+X94agTEZQIFluMhepSb1C8nkqUk8lRwUkgj2GAF1r6PsLXLpWHFpLisFvPKGDTmNFLND2xjKEZK0lf1MZwjNWkrrvqUvuj2BLsOS2pQUXkvVotzGhZp4XaYq+JRjTpJwSWvRdGUvFoQpUU4RS16LoylplE6xXL5CfGOdK/OipdR5Ftu+S7FG6svDW275Loux61KJvr5fg+Mc6D50Q6jyrbnyX0EqsskdufJfQ8l5RN5zl8seMc8hHnRM6jtHbbourJqVZZY7bdF1fYJKUTdPL5bnjHOsV50KlR35bLkuiFarLNy2XJfhXY8lZROrHLy6xz80J1Hne3uQqVZeI9vcvoYiUTqPC5vy1+T0Xo9IVG66Wn3ui6nrTqt4lLT7y5Lr6HY7V0Sk0ICkMlKg6zQhxyo45AwN7A9sdGqFKLuor3I6yOHoxd4wSfoibatjtgN7T3Oo/SJg7/WR6nsa7UsdsOyu09i+r9If+TvnrMPZAFQ7q9moRwahdNdbynnldX5SzSAKlpjIuO2jNlpF664L20lOaa+EewwBWZkJTtE0UN3QRAF90Sm3H5RVa3heSFU7MPv8AqgBdJzmC5acWUls3q+WnydmAN0s0pWxIpKGVKqpavQAqgVj0QA8s2y22ATmo4qUfZACy2dKko2Wttf0Ae0iAKlOTTqjfWbyyfoHRSAGLFhOFN52jaCK1JFfoGMAeWDZTa3Vh3FAGycccezsgCxWhbDTaNW2MAMBj/eAK/N2oojA0gDuejSqycsTmWGv5EwAaQc0n18v7w1AmIygQRLVk0PMrbdpcUkgk7t96u4jOvZGE4KcXGWxhUhGcHGWzOHOoAUpIUFBKiAoZKANAodhzjk6sVGbSdzhq0FCbindIxjzPMIA3Sco46sNtIK1ncPvJyA7THrSozqu0Ue1HD1K0rQVzoWjugSEUXNUcVmGxyE+nyz6cOyLzDcOhT1nqzpMJwqnS81TV/AuqUgCgFAN0WRbHsAEAEALdIOaT6+X94agTEZQIFOkSG1JaDwQWy8m8FgFPJVSoVhnT20gBTPSNna1ijcpS+quyz1S88OmkAapmzLNVNMgsyZTqXyRcZIrfl6HKlaFX1xDSe5DinuU3uhyEkiYbDLUslJaxuIaAJvK8kZxVcTzRUcmm+3sKTjGeKh4d1vt7OhUW2WL69lrwdyOgxVuVbKtXz6lNKdfItXz6nkuzL3nKpa5YpgjyEf7xM5VrR1e3fqyZzr5Y6y279WEkzL3TVLXLXuR1iqfVCpKtfd7Lr0Qqzr5tHLZdeiOr6M2VZhkWFLYkisspJKm2Som7mSRWsdJTinBXXJHXUoJ04trWy/Q1zVlWb8GrUGJPWcEJB1bN69qq1rStaxnlXQ9MkegztmRs7V7LcpXWNZJZy1qK7uisZGR7achZ1G7rUpzqK0QzlXHdAGNp2fZ+slqNSlNcb1ENZah/PDKtPqgLlS7qUnJp4Pqm5cc5W4lsdXStB6YC5VdPZYrn5vaKQHPpN0YQBVZBu66hSxeSlQJHYDjAHYbLm0qaBaOwoDD+0Aa5qzWnVpWtN4pyz3QBhaFpNMJ2zToABNfogCiaRaROO4DZRXBPTniTSAEUg2q9U+wQBepWW4OgEpvzCsQNyQRhjlXCAIbrdTffVfV0ZAfRAEabtMkXUYQBAunMmAA0gD6E0Z+Jy3qGv5EwAaQc0n18v7w1AmIygQVDujSsytkFoktCutQnlEbj2pG9P3xo4+FWVP8Al+0reJwrSpfyvacvBjmjkQJgk3oiUm3ZFp0d0Jefot6rLXaNtY7AeSO0/RFrhuGyl5qmiLnCcIlPzVdF0Ok2VZTMui4ygJG871HpUcyfTF3Tpxpq0VY6KlRhSjlgrImxmegQAQArtS32GCEKUVOHJtAK1n/lTkO00Eec6sY6PfobuHwFaus0VaPOT0XvfyIKJmff5DSJVG5TvGOHtuJISn0EmMM1WWyt6mw6WAofek6j6R8q971fsRHtCxVoShx2aeeVr5fAkJR8Ya8WgU+msTGm07uTZ5VsbCcfDp0oxXpd/wDk/lYtUexXiu3nEJDKnCAkPJqVZclWde2kAQJ60ZTWy9HGeWquKeqXAFS7pTzLjksGShdEPVuAGmLFK0iu4lCc6aUFfXl6FVxenUqUoqmm3fl6Mozl1DhvgJqhNLwpXFfTFPKlVjBXT3fyKCdCvGms0WtX17HsokLUspF4bOITXcegQ8GtKKtF8x9nrygssW9+TOldzubYbl3EuqbSrXHBVAeQ3uMX+CjKNCKkrPX9WdPw6MoYaMZqz139WObBtKVDaquNV18xmU5GYdp9UbRuhZNoynBkVcarc6U9sAQpy0JX4MWNY1e4GremtdSYA22zpHZxbomalSdY1k43kHUE7+isY549T2eGrL/B+5irugWzJPSZbYeYccLjVENqQpRo4kmiU4nCpjxxDzUpKOrN7htOVHFQnVWWK3bVls+bOazErtNcWeWfFnq19kU8aNWz8r26PqjrKmNwrlC1SO/4l+FirSiXpq9gjleAR5PZGxhaVRXumV/E8Vh55Mk099mux0S0LNQ7Nz6nE1Sl6pN4JoAhNcK1OYMXBx5TbTUhOy3ljtdOJ6YAz0Ytng7l1Z4pf1KJGOVchADS3tL8Lsv7VkenAAj0QAgDxVRSjVRgDY/YjywCEZnCpA+qsAWiTlGJah5TnTjh/bdAC60rXJUaHGAFDiirFR9kAYg9EATZay3F4kUEANJeykA5V6T/ALQB2ixRSXZA6pH8ogDTpBzSfXy/vDUCYjKBAQBQtJtBCtzWSl1N47aFGiRXwk0GHan6Iq8Vw5VJZoadSmxvClVlnp6PmN9HNDWZai18a75Shgk+Ynd6c42MPgqdHu+ptYXh9Kgr7vqWaNw3wgAgCFalqtS6b7ywkHADMqPQlIxUewRhOcYK8jYw2Fq4iWWkr9ei7t7IU0m5sZmUYP79Y+5r6z6I8vPU7L4/sb3+lwb/AO5P/gn/AO3wXqNLLsZiXB1TYBPKUcVq/WWcT7Y9YU4w2Rp4nGVsQ/5krrkuS9FsifGZqi3SDmk+vl/eGoExGUCBXbrwQGVEKIDyeSlSzyVDBKAVH2CAIM9bbZdlzdfwcV+izPVLHV4wBg/bTfC2TdfwYfHxWZriuX3auu7P0dMAVTTxSpiZQWWphd1oXu93xSq1UwUgHcfois4lh6lZRyK9r/Ip+LYWrXUPDV7Xvt26k7uezXB0voeamEqK0kDg0wcLtK7LZplHtgKU6VLLNWdz34ZQnRoZais7sfWZbbYcmtl/F4H4rMnxDAxo3hlvjdLE8sC2mw0qqX+fmDhKzJzmHTub+rdAECztN5FDCUKdWFJSQRqH8CKgjm48nXpp2cl7zajgcTKKlGm2n2ZAntNJJVmraDqisyikAal/FRaKQK3KZ74jx6b0zK/qSsFiEs7pu297PYoNpWRM6s97TGJSMWHRiVJAHJiop4SsnrH4o7GfGME7Wqc1ylyafQk6K2ZMMTku69LTCEJcxUWHTmkgYBNTiY2sNh6kKicloeXG+L4PEYOVOlO8nbSz6rqjqlqW02XJXZfwfUfisyP0d8YVbxzyEWZxBUO6taiF8Gol4U1nKYfT1flIFYAqPdDtIpmppoIGL5N6uOKEilIARzUsUIbwzRXA1zJgCIMjWAPLOllPuhCcTX6O2ALi2lEuQ20NY/4StyfpwwMAevTRQCpxd5Z5R7OgAQAotFMylQC0XEkVFSk1ByOEAaThlAEqVs5a8Tgnp/2gB5K2YlGQ9sATCQIAxL8AdXsU97s+qR/KIA1aQc0n18v7w1AmIygQEAEAEAEAEAIrRt1RWWJRAee8JVeLZ7VqG/zRjHjKq75Yav8AQs6GBioeNiXlhyXOXovm9DOytHwheufWX5g+MVkjzW05IHoxhClZ5pav/wC2McTxB1IeFSWSn0XPvJ82Oo9iuCACAFukHNJ9fL+8NQJiMoECy3HCnUkJUsh5OymlTsqyvED64Ahz9oL1sv3s9ziupx4pz5yAMH59fC2TwZ7mH8OJqduWx5z/AMrAGTVoL4U4eDPcy1hxPlvY85AHsraC9e93s9k3hxOGCvnIAxsy0F6ya72eNXx1OHe7GB4z2wAms/TaXYDjTqHgtL79RdSaXn3FAVCqZER4TxVKEssnr7Tfo8MxVaCqU4XT21XpzZSG7AnHkl5qVcU24VLQq+wKpWSpJoXARgRgRFdUwdSc3NbPU6PD8Xw1CjGjNtSilF6c1oyMrRudEtrDKruBm8VX2OSEVrTWVy7KxMcFVVRS5Xv8TGpxnCywzppu+Vrbnax1u259eq+LPDjGep65HzkW5x57atoLo33s8OOR1PT6yANdqT69bK97PCj6upx73fwHGe32QBUO6xOKVwarDqed5Wrx5vyVmAKVp80FWjMg9Zhv8EboAWLdLcuWnUkFKqorniKio3DfAEFhQXlmcKQBZbIlNSzdTg64TrFHwEbgOk74A8dmEMpuIxJzO81gBera2l+wQBuW4t0gVKiBh6BgIAa2bZiRio4/UIAchAA/vAGpwHGkAaU9FKwBvQgGAOr2N8XZ9Uj+UQBp0g5pPr5f3hqBMRlAgIAIAIA1zD6UJK1qCUpFSomgA6STENpK7M4QlUkowV29kiuF96eqGipiV3uZOPDzPIQfKzO6PC8qu2kevX0Lbw6OA1qWnV/Dyj/u6vty5j6zpBthAbaQEJG4b+kk7yekx7RioqyKyvXqV5udR3ZJjI8QgAgAgBbpBzSfXy/vDUCYjKBAsttRGpKU3jrk0FQK7K95wgCJPzTutl+9zzivGIx4pztgDB+ad4WzxBrqH8L6PLlu2AMmpp7hTne5rqWsNYjy3u2APZWae173e5yb8YjoV2wBXntOEyszNNuSzpVrUk3VNkCsuxQVKh0fXGtVxVOlLLIs8JwmviqfiQtbbV/sVqX0fmp4rm2W0ht110pC3AFCjq0kEAEZg5GNSrhJV5eJF6O36FvheKU8BTWGqxblG6drW3b01XUvmjK3m5FhssE3GUpJDiKG6mlcYsoqySOarTVSpKa5tv3s0zky78GLGoNOBnG+jqTjSsZHkMLbmndV8XI4xnxiOuR2wAWrNPUb73I45HjEdPpgDC1Jp3WyvEEceqm2jHvd/DOAKh3WJhw8GvMlPO+Gk15voMAUfS28q1psJNCHQUnoISmkAa9MJkFLTQRxpAU46VVK6il2lKACm6AJ9jWczKJS8pV5ZSCE0wSTjWogBXa1uOPLqhOFRU5Z1wygDwUTnir7oA3ykktw1y7YAsLMolCaD/cwBk7LZQAFZ3wBmDWABSRAHtaiAOsWN8XZ9Uj+UQBp0g5pPr5f3hqBMRlAgIAIAgWva7cukFZJUrBDaRVbh6EpGJ9OQjCdRQWptYXCVMRK0dlu3oku7FctZLkyoPTtAkYtywNUI6C4cnF5dg3R5Km5vNU9316m5UxdPDRdLCb8583/ALfwr4ssQEbBUnsAEAEAEAEALdIOaT6+X94agTEZQIFdulYDJbSlStcmgUopB2VZqCVEYdhgCDPvTetl6ssV1iqd8OY8Uv8A4fCAMH3pvhbPEsV1D9BwhdKX5auOo9G7pgBPpDpVMScztyzSy4ynATK6AIW5vLG+99Ua+IxMaNrrcseH8OnjM2VpZbb97/QTsd0p0OuK4G3tBGHCVYUB+YjXfEIWvZlgv+nqrk1nWnqR5aw5m03H5xAZaC3QLhcWqhQ00jlBsVrSuQzjGeHWKtVTt7Oj9TOnxGXDL4WUMzTvfNbdJ9GW3QpiaYlEtBthYQ6+LxfWmp4Q7XDUHfXfG/Thkgo9CixNfx60qlrXd7E2yXprgyKMsUufKHO3dweMzwIM49NfBi+JYu8DOOvXWmpONNRn2V9sAT7aem9WKssAaxrKYcPjkU/R4A9tV6bo3VljnUfpDmdf2eAMLUem9bK1ZY59VO+HMTwd/PvfDCvTAFQ7qzkweDX2mk87S68tVebrmyKQAv0okENzkw6o0vuVP0Af2gBHalpNKACUAkZKNd1YAWNTBK0qViEqSSPKANSn2jCAG2kVtpmSkNtJaSkAAAk9O8jtgCPZ9mFQvKy+/sgCwy7YCQKU7IAySkCAMHVVgDWlEAepMAZQAZ74A6zYvxdn1SP5RAGnSDmk+vl/eGoExGUCDwmAEEzb6nVFqRQHVDBTp5pr0kctXmp+kR4Oq5PLT178kWtPARpRVXFvKuUf8pezku7JVj2Glol1xRefVynVZ/qpHgJ80fXGUKSjq9X1PDFY6VZeHBZaa2iv1fV92No9TRCACACACACACAFukHNJ9fL+8NQJiMoECu3QshkNqSlWuTQqSVAbKs0hSScO0QBCn2ZvWy9XmK6xVO93MOKX/wARjACLTC2ZqTfl1lTDhU2+kcUtAAvME+OVXIdG+NfE4jwYqVrlhw7AfbKjhmy2V9r/ADQjlmH7ZfWVOtMFhtAwZWsKDinOl0Upd7c41ko4yN3pbv19hZyc+DStFqefqrWt7X1NrHc1dLrieGI2QjHgysag/PxL4fC1rsxX/UNVSbyLX1LJojY8zLpfYRMMkIezVLrxKmml5B/DlU9kblKmqcFBcinxeJeJrSqyVm7fBJfImWAzNapVHmBx8xnLrOPCHa+PG+sehrGVkszXBkUeYpcy4O527+EQBBnGZr4MXxzF3gZw1C601Jwrr8+2nsgCfbTM1qxV5gjWNZS7g8cin6RAHtqszdG6vMc6j9Hczr+0QBhajM1rJWrzHPqp3uvA8Hfz4/HCvRAFQ7qzcwODX3Wlc7S6ytNObrm8awBYLc7njMy8t1b7wKjW6m5QeiqawBB/+J5br3//AK/yQB6e5TLde/8A/X+SAJEt3M5dBrrXT6bn9kwBPOhDVKa1wf8AT+WAPP8AQzPWufZ/LAHv+iGesc+z+EAYnQZnrXPs/lgAGgrPWufZ/LAGX+iGesc+z+EAef6HZ6xz7P5YA8OgzPWufZ/LAFmlGA2hCBiEJCQT5opAGi1pNTrdxCghQW2sKUkqHFuIcoUhSa1u0z3wJTsatTNdcx/Dr/zwGhCtWxX5hIQ680Ug1KQ06kK7FXZgVHZGE4KaszYw2Knh5OVPfq0nb0utGSJaRmG0hDbkuhIFAlMssAewPxklZWR5VKkqknKbbb5tm3UzXXMfw6/88SYaBqZrrmP4df8AngNA1M11zH8Ov/PAaBqZrrmP4df+eA0DUzXXMfw6/wDPAaBqZrrmP4df+eA0DUzXXMfw6/8APAaBqZrrmP4df+eA0NM1ITDgSlbzN0ONrN1hYJ1biXKAl40rdpWhpXKIJTQ3iTEjzsoHABeUkpUFBSaVBFRvBG87oAhuWQVFKjMPVSSU81gSCk+L6CYAhWtoi1MlBfdeWUXgnaQml67e5KBWt1OfRHnUpQqK01c2MPiquHk5UnZvTZP9bnlk6INSxWph15BWEhRvIVUJvFPKQcrysumFOlCmrQVicTi62JadWV7baJfpYnIsghSlCYeqqlTxWNMvFx6GsDVjlJUUzDwK1Xlc1iQlKK835KUjDogDyXsYoFETDwBUpXis1qK1HFveok+2APWbHKUhCZh4JAoBxWXtbrAGtdggtFkvvasouFNW+SRdpXV1ywrWsAbH7HKxRUw8RUHxWaSFA4N7iAYAHbIKqXph40IUOawIyODcADtjFRQVTDxKFXk81gSlSK835KlDHpgCFa2iTUzd17ry7lbu0hNL1K8lA6BnAFhgAgDh8+h0BSyyprWzz7aJ/hj9GFCZcSgrYSLoAoECpunCtKwBc0aWzSrTVLhDfB0vhg1U2lRJaDt8FToWVY8gNkEAm9gQAK3PaZzMwxOoUtBbXITDzS221N01a0oF1SnCtaaK5RQjEYVxoBbu6PPqk0sT4KgllS0OgVIKXkFKSQMMHA0f/cAVHQ5cw263JOqdcMlrJpwqKqrC2Gy02Tv4xx6lerGEANUabTaWHHFOSzql2eucbDaTxBSU0bXtm+DeoFbNShYphAGdr2xNSj8w4442t1MjLY0UhlBenHm75TeOCEqBJriEbtwDjQp95U5PpefbeUjg4vNApRyFHkFSrqukV6IAuMAEAEAEAEAEAEAc1m5lxLjljhS77syFNrqqqZRwqed2q1F264zn4SOmAN9maYzS59bRbbEsl2YZAKm0rrLoUoKBLxcWVXa3dUKJUDUgYgL7Qt6cfs5TpnJVJdRLOpQ0DrGUuvoSULGs2k43SrZqUqFMcAGU/pXNNOucbLKTLOSzS27pDkyXwi86jbNwVWbqaLrcWK4QA80RtCZmFzK3lt6pEw+y2hKCFAMuqQFKWVEGoFKADp7IAssAEAEAEAEAEAEAEAEAU3ulNrWmSbQ3rSuboW9ctgLAl5hVC4ipSKgHLGggCs2BarkuxPOp72Ul1lkSrjj0yWXCq7f2qFWtCkFIBCcBiMYA3nTWeMq2pOqDvCH218WhSylkAgpYExtUrtXFrKaZGsAenSWZQuanQ824gWdLPJaCFhJKw9ikrUCkBQKiSkEpKQaXakCVKaT2gttpviEPrnUsXyELFxUu49eU0y+u6oFIoCvEUOFcAOiNg0F41NBUgUqd5pugDKACACACACAIi7MZLamS02WllRUgoSUqKlFSiU0oSVEk9JNYAxTZEuHQ+GGtcE3Q5cTfCQKXQqlaU3QBHTo3Ji/SUYF+9fo0gX7+Cr2GNd9c4Amz0i082Wnm0ONqpeQtIUk0IIqk4GhAPsgD1uSbS4p1LaA4sAKWEgKUE8kFWZA3QBEZsCVQHEolmUperrQG0AOVrW+ANrM59MASXrPaWVKW0hRWjVqJSCVIBJCDUYpqVGmWJgDXZtkS8uCJdhpkKpXVoSit3Kt0CtKmAJsAEAEAEAEAEAEAaFSTZcDxbTrQkpC7ovBJNSkKzpXGkAakWSwHjMBlsPKFC6EJvkYYFdKkYDfugDS3o/KJS6lMsyEvc6A0gBzftim1iTnAHosKVC23BLM32khLStWiraUigSk0qkAYACAJctLIbBDaEoClKUboAqpRqpRpmSaknfAG6ACACACACACACACACANT0shZSVJSooVeQSAbqqFN4VyNCoVG4mAI0zY0u4VlxhpZcSEuFSEm+lOISqo2gNwMAR3NGJJTeqVKS5bvBVwtIKbwSEBV2lKhICa9ApAElVkS5UFlhorCC2FXE1CDgWwacnzcoAxlbFlmkoQ2w0hKF30JS2lIQsgpvpAGCqKUKjcTAE+ACACACAP/2Q==", width=330)

Working with vector data like points or a bounding box requires a new library - `shapley`.  So now our two main spatial libraries are:
* `shapely` - vector data, such as points and polygons
* `rasterio` - raster data, such as satellite imagery

## Reprojecting vector objects

### The Reproject Function

This code is a custom function that we are defining and calling `reproject_vector`.  It uses some less-common functionality so I am not going to dig into the inside of this function.  For now just know that once you run this cell you can call `reproject_vector()` as a function for the rest of the notebook.

In [3]:
import pyproj
from shapely import ops
from functools import partial

def reproject_vector(geom, from_proj=None, to_proj=None):
    """
    Recommended function provided by shapely for projection transformations
    :param geom: shapely geometry object
    :param to_proj: source projection string
    :param from_proj: destination projection string
    :return: reprojected shapely geometry
    """
    tfm = partial(pyproj.transform, pyproj.Proj(from_proj), pyproj.Proj(to_proj))
    return ops.transform(tfm, geom)

## Reprojecting a Point or Bounding Box

**Step 1 - Define the shape we want to reproject**

Let's pick a shape to reproject.

For these examples I am going to reproject three objects:
1. the bounding box of the raster, which is a polygon
2. the geometric center of our raster (the **centroid**), which is a point
3. a random point I define myself

In [4]:
import rasterio
from shapely.geometry import box

In [5]:
filepath_rad = '../input_data/f100520t01p00r08rdn_b/f100520t01p00r08rdn_b_sc01_ort_img'
with rasterio.open(filepath_rad, 'r') as src:
    bbox = src.bounds

In [6]:
utm_bbox = box(*bbox)
utm_centroid = utm_bbox.centroid
print('Centroid: ', utm_centroid)
print('Bounding box: ', utm_bbox)

Centroid:  POINT (490183.9700000001 3357375.75)
Bounding box:  POLYGON ((504582.17 3350578.5, 504582.17 3364173, 475785.77 3364173, 475785.77 3350578.5, 504582.17 3350578.5))


The object that is returned from from the `.centroid` method is a shapley `Point` object.  The object produced in with the `box()` method is a `Polygon`.

Alternatively, we could use some other random point, but we will have to create that `Point` object ourselves.

In [7]:
from shapely.geometry import Point

In [8]:
# utm_point = Point(500000, 0)
utm_point = Point(475785.77, 3350578.5)

Notice that I chose a point which was at the center easting and sits on the equator (assuming a northern hemisphere grid).  We should be able to check that in our output.

**Step 2 - Define the input and output coordinate reference systems**

The source crs we can pull directly from the raster.  For the output since we want latitude and longitude as a result we can manually specify `EPSG:4326`.

In [9]:
with rasterio.open(filepath_rad, 'r') as src:
    src_crs = src.crs
print('source crs is: ', src_crs)

source crs is:  EPSG:32616


In [10]:
dst_crs = 'EPSG:4326'
print('destination crs is: ', dst_crs)

destination crs is:  EPSG:4326


**Step 3 - Input the shape into our reproject function**

Once our point is defined we can input it into our reproject function.

In [11]:
wgs84_point = reproject_vector(utm_point, from_proj=src_crs, to_proj=dst_crs)
wgs84_centroid = reproject_vector(utm_centroid, from_proj=src_crs, to_proj=dst_crs)
wgs84_bbox = reproject_vector(utm_bbox, from_proj=src_crs, to_proj=dst_crs)

In [12]:
print('random point in 4326: ', wgs84_point)
print('centroid in 4326: ', wgs84_centroid)
print('bounding box in 4326: ', wgs84_bbox)

random point in 4326:  POINT (30.28667237000204 -87.25178719598659)
centroid in 4326:  POINT (30.34821418934487 -87.10213411955947)
bounding box in 4326:  POLYGON ((30.28690584349906 -86.95235296598685, 30.40958435532548 -86.9522934966751, 30.40934973837654 -87.25210145321883, 30.28667237000204 -87.25178719598659, 30.28690584349906 -86.95235296598685))


# Reprojecting a raster

### Defining the Function

Reprojecting a raster is a bit more complicated than reprojecting vectors.  If you are interested in the nuts and bolts certainly take the time to walk through this function step by step.  Otherwise I would treat this function like the vector `reproject_vector` function, as a cell of code where you supply the inputs.

The code I have below is just a function of the code supplied in the [rasterio docs](https://rasterio.readthedocs.io/en/latest/topics/reproject.html).

In [13]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [14]:

# Raster reproject function
def reproject_raster(input_filepath, output_filepath, dst_crs):
    """
    Recommended function provided by rasterio for raster transformations
    :param input_filepath: the absolute or relative filepath of the raster to reproject
    :param output_filepath: where to save the reprojected file
    :param dst_crs: the crs to which to reproject the raster
    """
    with rasterio.open(input_filepath) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_filepath, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

### Running the Function

In [15]:
# Make the output directory if it does not exist yet
import os
if not os.path.exists('../output_data'):
    os.makedirs('../output_data')

In [16]:
# Define our inputs
filepath_rad = '../input_data/f100520t01p00r08rdn_b/f100520t01p00r08rdn_b_sc01_ort_img'
output_path = '../output_data/ort_img_4326'

# Run the function
reproject_raster(filepath_rad, output_path, 'EPSG:4326')

If we now open up our new raster we can see the output file.  Notice how the bounds changed to use our new coordinate system!

In [17]:
with rasterio.open(output_path, 'r') as src:
    print(src.meta)
    rplot.show(src)

{'driver': 'ENVI', 'dtype': 'int16', 'nodata': None, 'width': 1724, 'height': 707, 'count': 224, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.000173874084932819, -0.0, -87.2521014532188,
       -0.0, -0.000173874084932819, 30.4095930403477)}


NameError: name 'rplot' is not defined

Also notice in the metadata that the height and width have changed from the original source data.  This is because when reprojecting raster data we actually change it a little when we resample.